In [1]:
import re
import time
import nltk
import spacy
import pytz
import Chatbot_Entity_date as ed

from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.util import filter_spans
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

from konlpy.tag import Hannanum
from pykospacing import Spacing
from konlpy.tag import Okt, Kkma
from soynlp.normalizer import repeat_normalize
from nltk import word_tokenize, pos_tag, ne_chunk

okt = Okt()
kkma = Kkma()
spacing = Spacing()
hannanum = Hannanum()
kst = pytz.timezone('Asia/Seoul') # 한국 표준시 (KST)

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.data.find('chunkers/maxent_ne_chunker')
except LookupError:
    nltk.download('maxent_ne_chunker')

try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')

ModuleNotFoundError: No module named 'Chatbot_Entity_date'

In [43]:
from konlpy.tag import Komoran
import re
import spacy
from spacy.tokens import Span

# Komoran 형태소 분석기 사용
komoran = Komoran()

def extract_stock_entities(text):
    """주어진 텍스트에서 주식 관련 엔티티를 추출하는 통합 함수입니다."""
    
    # 주식 관련 패턴 정의
    patterns1 = {
        "주가": r"주가|주식|종가",
        "증시": r"증시",
        "삼성전자": r"삼성전자|삼성|삼전|samsung",
        "애플": r"애플|apple",
        "비트코인": r"비트코인|bitcoin|비트|코인|coin",
        "PER": r"PER|per|주가수익비율|Price Earning Ratio",
        "PBR": r"PBR|pbr|주가순자산비율|Price Book-value Ratio",
        "ROE": r"ROE|roe|자기자본이익률|Return on Equity",
        "MC": r"MC|mc|시가총액|총액|시총|Market Cap",
        "경제지표":r"경제지표|국내총생산|GDP|기준금리|IR|수입물가지수|IPI|생산자물가지수|PPI|소비자물가지수|CPI|외환보유액"
    }
    print("patterns1 defined:", patterns1)

    # 주식 관련 예측 및 변동성 패턴 정의 (동사 포함)
    patterns2 = {
        "예상": r"예상|예측|전망|앞으로",
        "가격": r"가격|값"
    }
    print("patterns2 defined:", patterns2)

    # 패턴 통합
    combined_patterns = {**patterns1, **patterns2}
    print("combined_patterns defined:", combined_patterns)

    # 텍스트에서 패턴에 맞는 주요 키워드 추출
    def extract_main_keyword(text, pattern):
        match = re.search(pattern, text)
        if match:
            print(f"Matched keyword in text '{text}' with pattern '{pattern}': {match.group(0)}")
            return match.group(0)  # 매칭된 주요 키워드 반환
        return None  # 매칭되지 않으면 None 반환

    def clean_text(text):
        # Komoran으로 형태소 분석을 수행
        token_pos = komoran.pos(text)
        cleaned_tokens = [word for word, pos in token_pos if not pos.startswith('J')]
        cleaned_text = ''.join(cleaned_tokens)

        return cleaned_text

    # 사용자 정의 spaCy 파이프라인 컴포넌트
    @spacy.Language.component("custom_stock_entity_adder")
    def custom_stock_entity_adder(doc):
        new_ents = []
        print("Processing doc:", doc)

        for token in doc:
            print("Processing token:", token.text)
            # 형태소 분석을 통해 명사와 동사/형용사 추출
            token_pos = komoran.pos(token.text)
            print(f"Token POS: {token_pos}")
            
            # 품사별로 나눠서 명사 및 동사 추출
            noun_phrase = ''.join([word for word, tag in token_pos if tag in ['NNG', 'NNP', 'SL']])  # 명사
            verb_phrase = ''.join([word for word, tag in token_pos if tag in ['VV', 'VA']])  # 동사/형용사
            print(f"Noun phrase: {noun_phrase}, Verb phrase: {verb_phrase}")

            # 형태소 분석 결과와 원래 텍스트 보정
            noun_phrase_cleaned = clean_text(noun_phrase)  # 형태소 분석된 명사에서 조사를 제거
            original_text_cleaned = clean_text(token.text)  # 원래 텍스트에서 조사를 제거

            found = False  # 해당 단어가 패턴과 매칭되는지 확인
            for label, pattern in combined_patterns.items():
                if noun_phrase_cleaned and len(noun_phrase_cleaned) > 1:  # 명사가 있으면
                    main_keyword = extract_main_keyword(noun_phrase_cleaned, pattern)
                    if main_keyword:
                        found = True
                        new_ent = Span(doc, token.i, token.i + 1, label=label)
                        new_ent._.set("cleaned_text", noun_phrase_cleaned)
                        new_ents.append(new_ent)
                        print(f"New entity added: {new_ent}, Label: {label}")
                        break

                if verb_phrase:  # 동사/형용사가 있으면
                    main_keyword = extract_main_keyword(verb_phrase, pattern)
                    if main_keyword:
                        found = True
                        new_ent = Span(doc, token.i, token.i + 1, label=label)
                        new_ent._.set("cleaned_text", verb_phrase)
                        new_ents.append(new_ent)
                        print(f"New entity added: {new_ent}, Label: {label}")
                        break

            if not found:
                # 원래 텍스트에 기반해 패턴 매칭 시도
                main_keyword = extract_main_keyword(original_text_cleaned, pattern)
                if main_keyword:
                    new_ent = Span(doc, token.i, token.i + 1, label=label)
                    new_ent._.set("cleaned_text", original_text_cleaned)
                    new_ents.append(new_ent)
                    print(f"New entity added based on original text: {new_ent}, Label: {label}")
                    break

        # 최종 엔티티 설정
        doc.ents = new_ents
        print(f"Final entities in doc: {new_ents}")

        return doc

    # spaCy 모델 로드
    nlp = spacy.load("ko_core_news_sm")
    print("spaCy model loaded")

    # 확장 속성 등록
    Span.set_extension("cleaned_text", default=None, force=True)
    print("Span extension 'cleaned_text' set")

    # 기존 파이프라인에서 custom_stock_entity_adder 제거
    if "custom_stock_entity_adder" in nlp.pipe_names:
        nlp.remove_pipe("custom_stock_entity_adder")
        print("Removed existing 'custom_stock_entity_adder' from pipeline")

    # custom_stock_entity_adder 추가
    nlp.add_pipe("custom_stock_entity_adder", after="ner")
    print("'custom_stock_entity_adder' added to pipeline")

    # 텍스트 처리
    doc = nlp(text)
    print(f"Processed text: {text}")

    # 결과 출력
    entities = [ent._.get("cleaned_text") for ent in doc.ents] + [ent.label_ for ent in doc.ents]
    print(f"Entities found in text: {entities}")
    return list(set(entities))


In [44]:
# 주식 정보: PER, PBR, ROE, MC

def stock_information(text):
    # 엔터티 추출
    entities = extract_stock_entities(text)
    
    # 엔터티 레이블 추출
    entity_labels = []
    for _, label in entities:
        entity_labels.append(label)
    
    # 주식 종목 리스트
    stock_labels = ["삼성전자", "애플", "비트코인"]

    # 사용자가 요청한 주식 종목을 추출
    requested_stocks = [stock for stock in stock_labels if stock in entity_labels]

    # 주식 정보 엔터티 (PBR, PER, ROE, MC) 리스트
    info_labels = ["PBR", "PER", "ROE", "MC"]

    # 사용자가 요청한 주식 정보 엔터티를 추출
    requested_infos = [info for info in info_labels if info in entity_labels]
    
    # SQL 쿼리 생성 및 답변 생성
    if requested_stocks and requested_infos:
        # 각 주식 종목과 정보에 따라 쿼리와 답변 생성
        stock = requested_stocks[0]  # 하나의 주식 종목만 처리
        info = requested_infos[0]  # 하나의 정보만 처리

        # SQL 쿼리 생성
        date = "2024-09-01" # 특정 날짜
        query = ""
        if stock == "삼성전자":
            if info == "PBR":
                query = f"SELECT sc_ss_pbr FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "PER":
                query = f"SELECT sc_ss_per FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "ROE":
                query = f"SELECT sc_ss_roe FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "MC":
                query = f"SELECT sc_ss_mc FROM tb_stock WHERE fd_date = '{date}';"
                return query

        elif stock == "애플":
            if info == "PBR":
                query = f"MSELECT sc_ap_pbr FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "PER":
                query = f"SELECT sc_ap_per FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "ROE":
                query = f"SELECT sc_ap_roe FROM tb_stock WHERE fd_date = '{date}';"
                return query
            elif info == "MC":
                query = f"MSELECT sc_ap_mc FROM tb_stock WHERE fd_date = '{date}';"
                return query

        elif stock == "비트코인":
            if info == "PBR":
                return ''
            elif info == "PER":
                return ''
            elif info == "ROE":
                return ''
            elif info == "MC":
                return ''
    
    return ''

In [ ]:
text = "삼성전자 주가알려줘"
def stock_create_quary(text):
    entity = extract_stock_entities(text)
    print(entity)
    if "주가" in entity:
        stock_price_samsung = stock_price(text)
        print(stock_price_samsung)
stock_create_quary(text)

In [46]:
# 주식 가격: 언제 삼성전자 주식 얼마야?

def stock_price(text):
    entities = extract_stock_entities(text)
    
    # 엔터티 레이블 추출
    entity_labels = []
    for _, label in entities:
        entity_labels.append(label)
        
    # 주식 종목 리스트
    stock_labels = ["삼성전자", "애플", "비트코인"]

    # 사용자가 요청한 주식 종목을 추출
    requested_stocks = [stock for stock in stock_labels if stock in entity_labels]
    
    # SQL 쿼리 생성
    date = "2024-09-01" # 특정 날짜
    query = ""
    # "가격" 엔터티가 있는지 확인
    if "가격" in entity_labels:
        if "삼성전자" in requested_stocks:
            return "SELECT sc_ss_stock FROM tb_stock WHERE sp_date = '{date}'"
        elif "애플" in requested_stocks:
            return "SELECT sc_ap_stock FROM tb_stock WHERE sp_date = '{date}'"
        elif "비트코인" in requested_stocks:
            return "SELECT sc_coin FROM tb_stock WHERE sp_date = '{date}'"
        else:
            return "해당 정보가 없습니다."
    else:
        return "다시 질문해주시겠습니까?"

In [ ]:
# 예상 (주식, 비트코인 가격 예측 페이지 던져주기)

def predict_stock(text):
    #엔터티 추출
    entities = extract_stock_entities(text)
    
    # 엔터티 레이블 추출
    entity_labels = []
    for _, label in entities:
        entity_labels.append(label)
        
    # "예상" 엔터티가 있는지 확인
    if "예상" in entity_labels:
        return "주식/비트코인에 대한 예상값은 다음 링크를 참조하세요: https://www.example.com/stock-prediction"
    
# 테스트 예제
text = "비트코인과 애플의 다음 달 예상 값을 알고 싶어요."
print(text)
print(predict_stock(text))

In [ ]:
# 경제지표 (비교 페이지 던져주기)

def economic_indicator(text):
    # 엔터티 추출
    entities = extract_stock_entities(text)
    
    # 엔터티 레이블 추출
    entity_labels = []
    for _, label in entities:
        entity_labels.append(label)
    
    # "경제지표" 엔티티가 있는지 확인
    if "경제지표" in entity_labels:
        return "경제지표에 대한 자세한 정보는 다음 링크를 참조하세요: https://www.example.com/economic-indicators"
    
# 테스트 예제
text = "다음 달의 경제지표를 알고 싶어요."
print(economic_indicator(text))